## Load the whales_2011_2012.geojson and get 2011 data.

In [4]:
import json
import pandas as pd

# Load the GeoJSON data
with open('whales_2011_2012.geojson', 'r', encoding='utf-8') as f:
    geojson_data = json.load(f)

# Extract features
features = geojson_data['features']

# Create a dataframe from properties and coordinates
records = []
for feature in features:
    props = feature.get('properties', {})
    year = props.get('year')
    if year == 2011:
        coords = feature.get('geometry', {}).get('coordinates', [None, None])
        props['longitude'] = coords[0]
        props['latitude'] = coords[1]
        records.append(props)

df_2011 = pd.DataFrame(records)

print(df_2011.shape)

# Show first few records as a check
df_2011.head()


(3972, 11)


,species,scientificName,eventDate,year,month,day,basisOfRecord,datasetName,occurrenceID,longitude,latitude
0,Tursiops truncatus,Tursiops truncatus,2011-07-29,2011,7,29,HumanObservation,Happywhale - Common bottlenose dolphin in Nort...,586003,-1.88030,48.706190
1,Tursiops truncatus,Tursiops truncatus,2011-03-15T11:56:43,2011,3,15,HumanObservation,Happywhale - Common bottlenose dolphin in Nort...,586191,-1.86016,49.433640
2,Tursiops truncatus,Tursiops truncatus,2011-03-15T12:38:51,2011,3,15,HumanObservation,Happywhale - Common bottlenose dolphin in Nort...,586101,-1.85414,49.434990
3,Tursiops truncatus,Tursiops truncatus,2011-06-26T19:09:30,2011,6,26,HumanObservation,Happywhale - Common bottlenose dolphin in Nort...,573489,-1.75326,48.846509
4,Tursiops truncatus,Tursiops truncatus,2011-07-04T14:44:02,2011,7,4,HumanObservation,Happywhale - Common bottlenose dolphin in Nort...,585981,-1.89430,48.895710


Ali sta prva dva stolpca enaka?

In [5]:
# Are 'species' and 'scientificName' in this dataset the same?
same_species = (df_2011['species'] == df_2011['scientificName'])
print(f"Number of records where they are equal: {same_species.sum()} out of {len(df_2011)}")
print(f"Are all the values the same? {same_species.all()}")
if not same_species.all():
    print("Examples where they're different:")
    diff = df_2011.loc[~same_species, ['species', 'scientificName']].drop_duplicates()
    print(diff.head())

Number of records where they are equal: 3972 out of 3972
Are all the values the same? True


Podatke lahko malo spucamo za našo uporabo:

- Prva dva stolpca ('species' in 'scientificName') sta enaka,
  zato lahko izbrišemo stolpec "species".
- Stolpca "eventDate" ne potrebujemo,
  ker imamo podatke že posebej v stolpcih "year", "month" in "day".
- "basisOfRecord" prav tako ni potreben, saj so vsi podatki rezultat opazovanja ljudi.
- Stolpec "datasetName" ne potrebujemo, ker že poznamo vir podatkov.
- "occurrenceID" je unikaten ID za opažanje in ga za analizo ne potrebujemo.

Torej odstranimo stolpce: "species", "eventDate", "basisOfRecord", "datasetName", "occurrenceID".

In [6]:
df_2011 = df_2011.drop(columns=["species", "eventDate", "basisOfRecord", "datasetName", "occurrenceID"])

df_2011.head()

,scientificName,year,month,day,longitude,latitude
0,Tursiops truncatus,2011,7,29,-1.88030,48.706190
1,Tursiops truncatus,2011,3,15,-1.86016,49.433640
2,Tursiops truncatus,2011,3,15,-1.85414,49.434990
3,Tursiops truncatus,2011,6,26,-1.75326,48.846509
4,Tursiops truncatus,2011,7,4,-1.89430,48.895710


Imamo 3972 podatkov za leto 2011. <br>

Prva dva stolpca sta ENAKA, zato lahko enega droppamo! <br>

Poglejmo, katere vrste kitov imamo:

In [7]:
whale_species = df_2011['scientificName'].unique()

print(f"Number of whale species = {len(whale_species)}")
print(whale_species)

Number of whale species = 30
['Tursiops truncatus' 'Stenella coeruleoalba' 'Delphinus delphis'
 'Physeter macrocephalus' 'Balaenoptera physalus' 'Grampus griseus'
 'Balaenoptera musculus' 'Orcinus orca' 'Phocoena phocoena' 'Delphinidae'
 'Globicephala melas' 'Stenella' 'Kogiidae' 'Cetacea'
 'Ziphius cavirostris' 'Phocoena phocoena relicta' 'Odontoceti'
 'Balaenoptera acutorostrata' 'Balaenopteridae'
 'Lagenorhynchus albirostris' 'Hyperoodon ampullatus'
 'Globicephala macrorhynchus' 'Leucopleurus acutus'
 'Tursiops truncatus ponticus' 'Megaptera novaeangliae' 'Delphinus'
 'Ziphiidae' 'Lagenorhynchus' 'Balaenoptera borealis'
 'Pseudorca crassidens']


# 1. Dataset: vsi podatki opažanj (all occurences) za kite v letu 2011. Flat array, samo vsi podatki

Imamo 30 vrst kitov. Izluščimo 3 vrste, ki imajo največ occurencev v našem datasetu:

In [8]:
species_counts = df_2011['scientificName'].value_counts()
species_counts_df = species_counts.reset_index()
species_counts_df.columns = ['scientificName', 'count']

print(species_counts_df[0:10])


               scientificName  count
0           Phocoena phocoena   1132
1       Stenella coeruleoalba    695
2           Delphinus delphis    531
3          Tursiops truncatus    340
4                 Delphinidae    195
5                   Delphinus    131
6  Balaenoptera acutorostrata    121
7       Balaenoptera physalus    110
8      Megaptera novaeangliae     96
9  Lagenorhynchus albirostris     93


**PAZI**: prvih nekaj z največjimi counti so DELFINI in ne KITI!

Te kiti imajo največ podatkov v databasu:

<ul>
<li>Balaenoptera acutorostrata</li>
<li>Balaenoptera physalus</li>
<li>Megaptera novaeangliae</li>
</ul>


Spodnja koda shrani json datoteko, ki vsebuje VSE occurance za kite. To bo uporabljeno za vizualizacijo proti koncu, kjer overlayamo vse kite naenkrat na mapo.

In [9]:
# fileter only the top 3 occuring whale species
df_2011_filtered = df_2011[df_2011['scientificName'].isin(['Balaenoptera acutorostrata', 'Balaenoptera physalus', 'Megaptera novaeangliae'])]

# save the filtered dataframe to a new file, making sure JSON lines are comma-separated and the whole output is an array
with open('whales_2011_top3_all_occurances.json', 'w') as f:
    records = df_2011_filtered.to_dict(orient='records')
    f.write('[\n')
    for i, rec in enumerate(records):
        line = json.dumps(rec)
        if i < len(records) - 1:
            f.write(line + ',\n')
        else:
            f.write(line)
    f.write('\n]')

print(df_2011_filtered.shape)

df_2011_filtered.head()

(327, 6)


,scientificName,year,month,day,longitude,latitude
41,Balaenoptera physalus,2011,7,12,6.774000,42.909833
44,Balaenoptera physalus,2011,9,8,7.144833,42.915167
45,Balaenoptera physalus,2011,9,8,7.030000,42.901667
64,Balaenoptera physalus,2011,7,1,7.220333,43.485000
69,Balaenoptera physalus,2011,9,9,7.180000,42.903333


# 2. Dataset: Kiti agregirani po imenu, nato po mesecih

Imamo 327 videvanj kitov v letu 2011. Sedaj agregiramo podatke po njihovem imenu. Ko shranimo json podatke, si želimo dostopati tako:

```json
{
  "Balaenoptera acutorostrata": {
    // next layer of data aggregates the data inside the species into months.
    "january": {
      // This is mean of all the whales in this species that were spoted in january (month = 0)
      // this variable is used to move and center the camera into the center of all whales of species spoted on this month
      "centroid": {"lon": float32, "lat": float32},

      // one entry for each occurence for this species and for this month
      "occurences": [
        {"lon": float32, "lat": float32},
        {"lon": float32, "lat": float32},
        {"lon": float32, "lat": float32},
        // ...
      ],

      // TODO: to compute the mean of both salinity and temperature, we will call an api call to our server, but for now, lets set it to 0.0.
      "mean_temperature": float32,
      "mean_salinity": float32
    },
    "february": {

    },
    
    // ... "march, "april" ... have one dictionary for each month (from january to december)
  },
  "Balaenoptera physalus": {
    // data for Balaenoptera physalus should have the same structure as the data for "Balaenoptera acutorostrata"
  },
  "Megaptera novaeangliae": {
    // data for Megaptera novaeangliae should have the same structure as the data for "Balaenoptera acutorostrata"
  }
}
```


Pripravimo si 2 helper funkciji: 
1. Vrne mean temperaturo preko lokalnega serverja, ki nam streže te podatke
2. Pretvori številko v čas, ki ga sprejme server

In [27]:
import requests


def fetch_mean_from_server(
  lon_lat_list,
  variable="thetao",
  time="2011-07-01",
  margin=1,
  depth=0,
  server_url="http://127.0.0.1:8000/mean",
):
  """
  Calls the server to fetch the mean value of specified variable for given lon/lat points.

  Args:
      lon_lat_list (list of dict): List of {"lon": float, "lat": float} for coordinates.
      variable (str): "thetao" for temperature or "so" for salinity.
      time (str): YYYY-MM-DD format date string.
      margin (float): Margin around each point (in degrees, probably).
      depth (int): Depth to query for variable.
      server_url (str): The URL of the mean API endpoint.

  Returns:
      The parsed JSON result (usually a float or dict, depending on the server response).
  """
  params = {
    "variable": variable,
    "margin": margin,
    "time": time,
    "depth": depth,
  }
  response = requests.post(server_url, params=params, json=lon_lat_list)
  response.raise_for_status()
  return response.json()


# Example usage:
# coords = [{"lon": -10, "lat": 10}, {"lon": -5, "lat": 15}]
# mean_temp = fetch_mean_from_server(coords, variable="thetao", time="2011-07-01")
# print(mean_temp)


def convert_number_to_time(number):
  """
  Convert a number to a time string in YYYY-MM-DD format.

  Args:
      number (int): The number to convert to time.

  Returns:
      str: The time string in YYYY-MM-DD format.
  """

  # this uses a day in the middle of the month, as we only have monthly data. Day is not relevant.
  return f"{2011}-{number:02d}-{15}"


convert_number_to_time(1)

'2011-01-15'

### THIS STEP TAKES ~ 80min!!

As we fetch local square of data for salinity and temperature for each occurence (around 360) this fetching takes a long time. Only compute when neccessary.

In [33]:
import json
import numpy as np

# Helper to map month number to lowercase month name
month_names = [
  "january",
  "february",
  "march",
  "april",
  "may",
  "june",
  "july",
  "august",
  "september",
  "october",
  "november",
  "december",
]

result = {}

# Use only species that occur in the current dataframe
species_list = df_2011_filtered["scientificName"].unique()
for species in species_list:
  species_dict = {}
  # iterate over all months in numerical order, as database is using 1-12 for months
  for m in range(0, 12):
    month_name = month_names[m]

    # Debug information: print what species and month is being computed
    print(f"Computing species: {species}, month: {month_name} ({m + 1})")
    # Filter rows corresponding to this species and month
    subset = df_2011_filtered[
      (df_2011_filtered["scientificName"] == species) & (df_2011_filtered["month"] == m)
    ]
    occurrences = [
      {"lon": float(row["longitude"]), "lat": float(row["latitude"])}
      for idx, row in subset.iterrows()
    ]

    # Calculate centroid if there are occurrences, otherwise set to None
    if len(occurrences) > 0:
      lon_mean = float(np.mean([o["lon"] for o in occurrences]))
      lat_mean = float(np.mean([o["lat"] for o in occurrences]))
      centroid = {"lon": lon_mean, "lat": lat_mean}
    else:
      centroid = None

    month_entry = {
      "centroid": centroid,
      "occurences": occurrences,
      # add one to the month, as m is in 0-11 range.
      "mean_temperature": (
        float("nan")
        if len(occurrences) == 0
        else fetch_mean_from_server(
          occurrences,
          variable="thetao",
          time=convert_number_to_time(m + 1),
          margin=1,
          depth=0,
        )
      ),
      "mean_salinity": float("nan")
      if len(occurrences) == 0
      else fetch_mean_from_server(
        occurrences,
        variable="so",
        time=convert_number_to_time(m + 1),
        margin=1,
        depth=0,
      ),
    }
    species_dict[month_name] = month_entry
  result[species] = species_dict

# Optionally: save to JSON file for later use
with open("whales_2011_top3_by_species_month.json", "w") as f:
  json.dump(result, f, indent=2)


Computing species: Balaenoptera physalus, month: january (1)
Computing species: Balaenoptera physalus, month: february (2)
Computing species: Balaenoptera physalus, month: march (3)
Computing species: Balaenoptera physalus, month: april (4)
Computing species: Balaenoptera physalus, month: may (5)
Computing species: Balaenoptera physalus, month: june (6)
Computing species: Balaenoptera physalus, month: july (7)
Computing species: Balaenoptera physalus, month: august (8)
Computing species: Balaenoptera physalus, month: september (9)
Computing species: Balaenoptera physalus, month: october (10)
Computing species: Balaenoptera physalus, month: november (11)
Computing species: Balaenoptera physalus, month: december (12)
Computing species: Balaenoptera acutorostrata, month: january (1)
Computing species: Balaenoptera acutorostrata, month: february (2)
Computing species: Balaenoptera acutorostrata, month: march (3)
Computing species: Balaenoptera acutorostrata, month: april (4)
Computing spec

Mean: POST /mean?variable=thetao&margin=5&time=2011-07-01&depth=0
Request body (JSON): [{"lon": -10, "lat": 10}, {"lon": -5, "lat": 15}]
Returns the mean value of the specified variable (thetao or so) computed across multiple coordinate points. For each (lon, lat) pair, creates a square region with the given margin (in degrees) around the point, computes the mean value in that region, then returns the overall mean of all computed means.

In [ ]:
import requests

# Example: Fetch mean temperature for coordinates
url = "http://127.0.0.1:8000/mean"
params = {
    "variable": "thetao",  # Use "so" for salinity
    "margin": 1, # 5 is a huge margin! Instead use 0.5 or 1 for not too big spatial square to compute mean
    "time": "2011-07-01",
    "depth": 0
}
#body = [{"lon": -10, "lat": 10}, {"lon": -5, "lat": 15}]

# Use real data, to see if it is fetching correctly
body = result["Balaenoptera physalus"]["february"]["occurences"]

response = requests.post(url, params=params, json=body)
result = response.json()
print(result)

{'mean': 17.293543301975586, 'points_processed': 2, 'total_points': 2}


# 3. Dataset: delitev kitov glede na morja

Europe: Top_left: (lat: 29, lon: -15), Bottom_right: (lat: 72, lon: 45) 

North Atlantic: Top_left (lat: 30, lon: -80), Bottom_right (lat: 75, lon: 20)

Json bo izgledal takole:

```json
{
  "Europe": [
    {
      "scientificName": string,
      "year": int,
      "month": int,
      "day": int
    },
    // next whale that falls into Europe bounds
  ],
  "North_Atlantic": [
    // Same as Europe format, but for whales that fall into North Atlantic bounds
  ]
}
```

In [10]:
# Define region bounds
# Europe: Top_left: (lat: 29, lon: -15), Bottom_right: (lat: 72, lon: 45)
europe_lat_min, europe_lat_max = 29, 72
europe_lon_min, europe_lon_max = -15, 45

# North Atlantic: Top_left (lat: 30, lon: -80), Bottom_right (lat: 75, lon: 20)
north_atlantic_lat_min, north_atlantic_lat_max = 30, 75
north_atlantic_lon_min, north_atlantic_lon_max = -80, 20

# Filter whales by region
europe_whales = df_2011[
    (df_2011['latitude'] >= europe_lat_min) & (df_2011['latitude'] <= europe_lat_max) &
    (df_2011['longitude'] >= europe_lon_min) & (df_2011['longitude'] <= europe_lon_max)
]

north_atlantic_whales = df_2011[
    (df_2011['latitude'] >= north_atlantic_lat_min) & (df_2011['latitude'] <= north_atlantic_lat_max) &
    (df_2011['longitude'] >= north_atlantic_lon_min) & (df_2011['longitude'] <= north_atlantic_lon_max)
]

# Convert to JSON format with only scientificName, year, month, day
result = {
    "Europe": europe_whales[['scientificName', 'year', 'month', 'day']].to_dict(orient='records'),
    "North_Atlantic": north_atlantic_whales[['scientificName', 'year', 'month', 'day']].to_dict(orient='records')
}

# Save to JSON file
with open('whales_2011_by_region.json', 'w') as f:
    json.dump(result, f, indent=2)

print(f"Europe whales: {len(result['Europe'])}")
print(f"North Atlantic whales: {len(result['North_Atlantic'])}")
print(f"Total unique whales in dataset: {len(df_2011)}")

Europe whales: 3610
North Atlantic whales: 3891
Total unique whales in dataset: 3972
